In [3]:
!pip install psycopg2-binary


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [6]:
!pip install psycopg2


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [15]:
from feast import FeatureStore

# Absolute path to your Feast repo (from inside the container)
store = FeatureStore(repo_path="/app/feast_repo")

# Quick test
print("Project:", store.project)
print("Feature views:", [fv.name for fv in store.list_feature_views()])

Project: feast_project
Feature views: []


In [17]:
# /app/feast_repo/customer_fv.py
from feast import Entity, FeatureView, Field, FileSource
from feast.types import Int64, String

customer = Entity(name="customer_id", join_keys=["customer_id"])

src = FileSource(path="app/feast_repo/data/customers.csv", timestamp_field="event_timestamp")

customer_features = FeatureView(         # <-- name MUST be exactly "customer_features"
    name="customer_features",
    entities=[customer],
    schema=[Field(name="segment", dtype=String), Field(name="age", dtype=Int64)],
    online=True,
    source=src,
)


In [18]:
customer_features

<FeatureView(name = customer_features, entities = ['customer_id'], ttl = 0:00:00, stream_source = None, batch_source = {
  "type": "BATCH_FILE",
  "timestampField": "event_timestamp",
  "fileOptions": {
    "uri": "app/feast_repo/data/customers.csv"
  },
  "name": "app/feast_repo/data/customers.csv"
}, entity_columns = [], features = [Field(
    name='segment',
    dtype=<PrimitiveFeastType.STRING: 2>,
    description='',
    tags={}
), Field(
    name='age',
    dtype=<PrimitiveFeastType.INT64: 4>,
    description='',
    tags={}
)], description = , tags = {}, owner = , projection = FeatureViewProjection(name='customer_features', name_alias=None, desired_features=[], features=[Field(
    name='segment',
    dtype=<PrimitiveFeastType.STRING: 2>,
    description='',
    tags={}
), Field(
    name='age',
    dtype=<PrimitiveFeastType.INT64: 4>,
    description='',
    tags={}
)], join_key_map={}, timestamp_field=None, date_partition_column=None, created_timestamp_column=None, batch_sourc

In [19]:
import os
from feast import FeatureStore
import pandas as pd

store = FeatureStore(repo_path="/app/feast_repo")

# Fetch online features for a few customers
entity_df = pd.DataFrame({"customer_id":[1,2,3]})
online_features = store.get_online_features(
    features=[
        "customer_features:total_orders",
        "customer_features:avg_order_value",
    ],
    entity_rows=entity_df.to_dict("records")
).to_dict()
online_features

FeatureViewNotFoundException: Feature view customer_features does not exist in project feast_project

In [14]:
import os
print(os.getcwd())

/app/notebooks
